In [1]:
import os
import zipfile
import shutil
import hashlib
from tqdm import tqdm_notebook
from keras import layers

C:\Users\JAMALOU\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_path = 'train.zip'
test_path = 'test_stg1.zip'
data_base_dir = 'data'

In [3]:
if not os.path.exists(data_base_dir):
    os.makedirs(data_base_dir)

In [4]:
def extract_zip(zip_path, out_dir):
    name = os.path.basename(zip_path).split('.')[0]
    zip_ref = zipfile.ZipFile(zip_path, 'r')
    zip_ref.extractall(out_dir)
    zip_ref.close()
    return os.path.join(out_dir, name)

In [5]:
orig_train_dir = extract_zip(train_path, data_base_dir)

In [6]:
test_dir  = extract_zip(test_path, data_base_dir)

In [7]:
training_dir = os.path.join(data_base_dir, 'train_val_split', 'training')
validation_dir = os.path.join(data_base_dir, 'train_val_split', 'validation')

In [8]:
classes = [class_ for class_ in os.listdir(orig_train_dir) if os.path.isdir(os.path.join(orig_train_dir, class_))]

In [9]:
classes

['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [10]:
for class_ in tqdm_notebook(classes):
    
    class_orig_dir = os.path.join(orig_train_dir, class_)
    class_training_dir = os.path.join(training_dir, class_)
    class_validation_dir = os.path.join(validation_dir, class_)
    
    if not os.path.exists(class_training_dir):
        os.makedirs(class_training_dir)
        
    if not os.path.exists(class_validation_dir):
        os.makedirs(class_validation_dir)

    img_list = os.listdir(class_orig_dir)

    for img in img_list:
        hash_name = hashlib.sha1(img.encode('ascii'))
        if int(hash_name.hexdigest(), 16) % 1000 > 100:
            shutil.copy(os.path.join(class_orig_dir, img), class_training_dir)
        else:
            shutil.copy(os.path.join(class_orig_dir, img), class_validation_dir)

KeyboardInterrupt: 

In [11]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [12]:
from keras.applications.vgg16 import VGG16

In [13]:
conv_base = InceptionResNetV2(include_top=False) #VGG16(include_top=False) #

In [14]:
from keras import layers, models, optimizers

In [15]:
model = models.Sequential()

In [16]:
model.add(conv_base)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(8, activation='softmax'))

In [17]:
conv_base.trainable = False

In [18]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-4), metrics=['accuracy'])

In [19]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpointer = ModelCheckpoint('quicksign_inception_resnet_512.h5', monitor='val_loss', save_best_only=True, verbose=1)
earlystopper = EarlyStopping(monitor='val_loss', patience=5)


In [20]:
from keras.preprocessing.image import ImageDataGenerator

In [21]:
train_data_gen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [22]:
train_generator = train_data_gen.flow_from_directory(training_dir,
                                                    target_size=(512, 512),
                                                    batch_size=16,
                                                    class_mode='categorical')
validation_generator = validation_data_gen.flow_from_directory(validation_dir,
                                                    target_size=(512, 512),
                                                    batch_size=32,
                                                    class_mode='categorical')

Found 3393 images belonging to 8 classes.
Found 384 images belonging to 8 classes.


In [23]:
model.fit_generator(train_generator, epochs=50, validation_data=validation_generator, verbose=1,
                    callbacks=[checkpointer, earlystopper])

Epoch 1/50


206/213 [============================>.] - ETA: 42:56 - loss: 2.1580 - acc: 0.12 - ETA: 22:34 - loss: 2.1154 - acc: 0.12 - ETA: 15:46 - loss: 2.1037 - acc: 0.14 - ETA: 12:22 - loss: 2.1210 - acc: 0.14 - ETA: 10:19 - loss: 2.1091 - acc: 0.15 - ETA: 8:57 - loss: 2.0848 - acc: 0.1354 - ETA: 7:58 - loss: 2.0692 - acc: 0.160 - ETA: 7:14 - loss: 2.0481 - acc: 0.148 - ETA: 6:39 - loss: 2.0337 - acc: 0.173 - ETA: 6:11 - loss: 2.0196 - acc: 0.168 - ETA: 5:49 - loss: 2.0162 - acc: 0.170 - ETA: 5:29 - loss: 2.0007 - acc: 0.171 - ETA: 5:13 - loss: 2.0065 - acc: 0.173 - ETA: 4:59 - loss: 1.9894 - acc: 0.187 - ETA: 4:47 - loss: 1.9707 - acc: 0.204 - ETA: 4:36 - loss: 1.9622 - acc: 0.218 - ETA: 4:26 - loss: 1.9569 - acc: 0.227 - ETA: 4:18 - loss: 1.9447 - acc: 0.236 - ETA: 4:13 - loss: 1.9302 - acc: 0.250 - ETA: 4:11 - loss: 1.9323 - acc: 0.250 - ETA: 4:10 - loss: 1.9170 - acc: 0.261 - ETA: 4:10 - loss: 1.9174 - acc: 0.269 - ETA: 4:08 - loss: 1.9099 - acc: 0.274 - ETA: 4:07 - loss: 1.9073 - acc: 0.27

213/213 [==============================] - ETA: 7s - loss: 1.6314 - acc: 0.434 - ETA: 6s - loss: 1.6309 - acc: 0.434 - ETA: 5s - loss: 1.6300 - acc: 0.434 - ETA: 3s - loss: 1.6307 - acc: 0.434 - ETA: 2s - loss: 1.6312 - acc: 0.434 - ETA: 1s - loss: 1.6309 - acc: 0.434 - 309s 1s/step - loss: 1.6292 - acc: 0.4343 - val_loss: 1.6757 - val_acc: 0.3750

Epoch 00001: val_loss improved from inf to 1.67566, saving model to quicksign_inception_resnet_512.h5
Epoch 2/50


206/213 [============================>.] - ETA: 2:27 - loss: 1.3097 - acc: 0.500 - ETA: 2:26 - loss: 1.3187 - acc: 0.562 - ETA: 2:26 - loss: 1.4170 - acc: 0.520 - ETA: 2:25 - loss: 1.4737 - acc: 0.500 - ETA: 2:24 - loss: 1.5013 - acc: 0.462 - ETA: 2:24 - loss: 1.4897 - acc: 0.510 - ETA: 2:23 - loss: 1.4667 - acc: 0.508 - ETA: 2:22 - loss: 1.4923 - acc: 0.500 - ETA: 2:21 - loss: 1.5404 - acc: 0.506 - ETA: 2:21 - loss: 1.5566 - acc: 0.487 - ETA: 2:20 - loss: 1.5413 - acc: 0.483 - ETA: 2:19 - loss: 1.5637 - acc: 0.479 - ETA: 2:19 - loss: 1.5459 - acc: 0.500 - ETA: 2:18 - loss: 1.5624 - acc: 0.491 - ETA: 2:17 - loss: 1.5516 - acc: 0.495 - ETA: 2:17 - loss: 1.5641 - acc: 0.488 - ETA: 2:16 - loss: 1.5858 - acc: 0.477 - ETA: 2:15 - loss: 1.5809 - acc: 0.479 - ETA: 2:15 - loss: 1.5972 - acc: 0.467 - ETA: 2:14 - loss: 1.5854 - acc: 0.471 - ETA: 2:20 - loss: 1.5902 - acc: 0.464 - ETA: 2:24 - loss: 1.5846 - acc: 0.463 - ETA: 2:29 - loss: 1.5884 - acc: 0.462 - ETA: 2:33 - loss: 1.5875 - acc: 0.463

213/213 [==============================] - ETA: 7s - loss: 1.5615 - acc: 0.456 - ETA: 6s - loss: 1.5615 - acc: 0.456 - ETA: 5s - loss: 1.5617 - acc: 0.456 - ETA: 3s - loss: 1.5637 - acc: 0.455 - ETA: 2s - loss: 1.5638 - acc: 0.455 - ETA: 1s - loss: 1.5652 - acc: 0.454 - 299s 1s/step - loss: 1.5647 - acc: 0.4545 - val_loss: 1.6245 - val_acc: 0.4271

Epoch 00002: val_loss improved from 1.67566 to 1.62451, saving model to quicksign_inception_resnet_512.h5
Epoch 3/50


...

213/213 [==============================] - ETA: 8s - loss: 1.3218 - acc: 0.548 - ETA: 6s - loss: 1.3223 - acc: 0.548 - ETA: 5s - loss: 1.3224 - acc: 0.548 - ETA: 4s - loss: 1.3233 - acc: 0.547 - ETA: 2s - loss: 1.3235 - acc: 0.547 - ETA: 1s - loss: 1.3238 - acc: 0.546 - 309s 1s/step - loss: 1.3230 - acc: 0.5475 - val_loss: 1.4932 - val_acc: 0.4375

Epoch 00014: val_loss did not improve from 1.44322
Epoch 15/50


206/213 [============================>.] - ETA: 2:27 - loss: 1.4650 - acc: 0.437 - ETA: 2:27 - loss: 1.3177 - acc: 0.531 - ETA: 2:26 - loss: 1.3218 - acc: 0.520 - ETA: 2:25 - loss: 1.3509 - acc: 0.515 - ETA: 2:24 - loss: 1.4401 - acc: 0.475 - ETA: 2:24 - loss: 1.3738 - acc: 0.510 - ETA: 2:23 - loss: 1.3575 - acc: 0.526 - ETA: 2:23 - loss: 1.3774 - acc: 0.515 - ETA: 2:22 - loss: 1.3724 - acc: 0.520 - ETA: 2:21 - loss: 1.3581 - acc: 0.525 - ETA: 2:21 - loss: 1.3628 - acc: 0.528 - ETA: 2:20 - loss: 1.3869 - acc: 0.510 - ETA: 2:19 - loss: 1.3810 - acc: 0.504 - ETA: 2:19 - loss: 1.3615 - acc: 0.517 - ETA: 2:18 - loss: 1.3749 - acc: 0.508 - ETA: 2:17 - loss: 1.3943 - acc: 0.500 - ETA: 2:17 - loss: 1.3800 - acc: 0.503 - ETA: 2:16 - loss: 1.3987 - acc: 0.500 - ETA: 2:17 - loss: 1.3807 - acc: 0.509 - ETA: 2:24 - loss: 1.3729 - acc: 0.518 - ETA: 2:31 - loss: 1.3797 - acc: 0.508 - ETA: 2:36 - loss: 1.3738 - acc: 0.514 - ETA: 2:38 - loss: 1.3602 - acc: 0.529 - ETA: 2:42 - loss: 1.3530 - acc: 0.531

213/213 [==============================] - ETA: 8s - loss: 1.3029 - acc: 0.554 - ETA: 6s - loss: 1.3040 - acc: 0.554 - ETA: 5s - loss: 1.3057 - acc: 0.552 - ETA: 4s - loss: 1.3058 - acc: 0.552 - ETA: 2s - loss: 1.3050 - acc: 0.553 - ETA: 1s - loss: 1.3043 - acc: 0.553 - 311s 1s/step - loss: 1.3059 - acc: 0.5531 - val_loss: 1.5193 - val_acc: 0.3724

Epoch 00015: val_loss did not improve from 1.44322
Epoch 16/50


206/213 [============================>.] - ETA: 2:28 - loss: 0.8540 - acc: 0.750 - ETA: 2:27 - loss: 1.1469 - acc: 0.593 - ETA: 2:26 - loss: 1.1486 - acc: 0.666 - ETA: 2:25 - loss: 1.1590 - acc: 0.609 - ETA: 2:25 - loss: 1.1864 - acc: 0.625 - ETA: 2:24 - loss: 1.1780 - acc: 0.635 - ETA: 2:23 - loss: 1.1900 - acc: 0.633 - ETA: 2:23 - loss: 1.1784 - acc: 0.648 - ETA: 2:22 - loss: 1.2019 - acc: 0.625 - ETA: 2:21 - loss: 1.2012 - acc: 0.625 - ETA: 2:20 - loss: 1.2227 - acc: 0.619 - ETA: 2:20 - loss: 1.2371 - acc: 0.609 - ETA: 2:19 - loss: 1.2631 - acc: 0.596 - ETA: 2:18 - loss: 1.2581 - acc: 0.598 - ETA: 2:18 - loss: 1.2480 - acc: 0.604 - ETA: 2:17 - loss: 1.2693 - acc: 0.593 - ETA: 2:16 - loss: 1.2767 - acc: 0.588 - ETA: 2:16 - loss: 1.2641 - acc: 0.597 - ETA: 2:15 - loss: 1.2802 - acc: 0.592 - ETA: 2:21 - loss: 1.2733 - acc: 0.590 - ETA: 2:25 - loss: 1.2754 - acc: 0.589 - ETA: 2:29 - loss: 1.2842 - acc: 0.585 - ETA: 2:34 - loss: 1.2927 - acc: 0.578 - ETA: 2:39 - loss: 1.3022 - acc: 0.572

213/213 [==============================] - ETA: 7s - loss: 1.3075 - acc: 0.551 - ETA: 6s - loss: 1.3066 - acc: 0.552 - ETA: 5s - loss: 1.3055 - acc: 0.552 - ETA: 3s - loss: 1.3057 - acc: 0.551 - ETA: 2s - loss: 1.3049 - acc: 0.552 - ETA: 1s - loss: 1.3057 - acc: 0.551 - 300s 1s/step - loss: 1.3063 - acc: 0.5508 - val_loss: 1.4512 - val_acc: 0.4401

Epoch 00016: val_loss did not improve from 1.44322


In [24]:
model.load_weights('quicksign_inception_resnet_512.h5')

In [25]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if 'conv_7b' in layer.name:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
    

In [26]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-5), metrics=['accuracy'])

In [38]:
model.fit_generator(train_generator, epochs=100, validation_data=validation_generator, verbose=1,
                    callbacks=[checkpointer, earlystopper])

Epoch 1/100


206/213 [============================>.] - ETA: 9:00 - loss: 0.4109 - acc: 0.937 - ETA: 5:44 - loss: 0.5154 - acc: 0.781 - ETA: 5:10 - loss: 0.4132 - acc: 0.833 - ETA: 4:59 - loss: 0.4311 - acc: 0.828 - ETA: 4:51 - loss: 0.4205 - acc: 0.862 - ETA: 4:46 - loss: 0.4128 - acc: 0.864 - ETA: 4:42 - loss: 0.4218 - acc: 0.866 - ETA: 4:39 - loss: 0.4432 - acc: 0.851 - ETA: 4:35 - loss: 0.4560 - acc: 0.840 - ETA: 4:30 - loss: 0.4521 - acc: 0.843 - ETA: 4:27 - loss: 0.4361 - acc: 0.858 - ETA: 4:24 - loss: 0.4310 - acc: 0.864 - ETA: 4:21 - loss: 0.4345 - acc: 0.870 - ETA: 4:18 - loss: 0.4209 - acc: 0.879 - ETA: 4:15 - loss: 0.4141 - acc: 0.887 - ETA: 4:13 - loss: 0.4228 - acc: 0.878 - ETA: 4:11 - loss: 0.4245 - acc: 0.878 - ETA: 4:09 - loss: 0.4269 - acc: 0.878 - ETA: 4:07 - loss: 0.4233 - acc: 0.878 - ETA: 4:04 - loss: 0.4452 - acc: 0.875 - ETA: 4:02 - loss: 0.4546 - acc: 0.869 - ETA: 4:01 - loss: 0.4475 - acc: 0.872 - ETA: 3:59 - loss: 0.4476 - acc: 0.875 - ETA: 3:57 - loss: 0.4432 - acc: 0.880

213/213 [==============================] - ETA: 7s - loss: 0.4552 - acc: 0.860 - ETA: 6s - loss: 0.4547 - acc: 0.860 - ETA: 5s - loss: 0.4548 - acc: 0.860 - ETA: 3s - loss: 0.4548 - acc: 0.861 - ETA: 2s - loss: 0.4553 - acc: 0.860 - ETA: 1s - loss: 0.4549 - acc: 0.860 - 295s 1s/step - loss: 0.4541 - acc: 0.8609 - val_loss: 0.5178 - val_acc: 0.8359

Epoch 00001: val_loss did not improve from 0.51281
Epoch 2/100


206/213 [============================>.] - ETA: 2:33 - loss: 0.5839 - acc: 0.750 - ETA: 2:32 - loss: 0.5124 - acc: 0.781 - ETA: 2:33 - loss: 0.4817 - acc: 0.812 - ETA: 2:32 - loss: 0.4952 - acc: 0.812 - ETA: 2:32 - loss: 0.5106 - acc: 0.825 - ETA: 2:31 - loss: 0.5000 - acc: 0.812 - ETA: 2:31 - loss: 0.4876 - acc: 0.812 - ETA: 2:30 - loss: 0.4696 - acc: 0.820 - ETA: 2:29 - loss: 0.4665 - acc: 0.826 - ETA: 2:29 - loss: 0.4484 - acc: 0.837 - ETA: 2:28 - loss: 0.4549 - acc: 0.840 - ETA: 2:28 - loss: 0.4379 - acc: 0.849 - ETA: 2:27 - loss: 0.4389 - acc: 0.846 - ETA: 2:26 - loss: 0.4491 - acc: 0.839 - ETA: 2:25 - loss: 0.4386 - acc: 0.845 - ETA: 2:24 - loss: 0.4322 - acc: 0.847 - ETA: 2:24 - loss: 0.4280 - acc: 0.852 - ETA: 2:23 - loss: 0.4199 - acc: 0.857 - ETA: 2:22 - loss: 0.4363 - acc: 0.855 - ETA: 2:21 - loss: 0.4386 - acc: 0.856 - ETA: 2:26 - loss: 0.4399 - acc: 0.857 - ETA: 2:31 - loss: 0.4354 - acc: 0.858 - ETA: 2:35 - loss: 0.4295 - acc: 0.858 - ETA: 2:39 - loss: 0.4368 - acc: 0.854

213/213 [==============================] - ETA: 7s - loss: 0.4528 - acc: 0.860 - ETA: 6s - loss: 0.4526 - acc: 0.860 - ETA: 5s - loss: 0.4517 - acc: 0.860 - ETA: 3s - loss: 0.4512 - acc: 0.861 - ETA: 2s - loss: 0.4505 - acc: 0.862 - ETA: 1s - loss: 0.4503 - acc: 0.861 - 302s 1s/step - loss: 0.4496 - acc: 0.8621 - val_loss: 0.5171 - val_acc: 0.8464

Epoch 00002: val_loss did not improve from 0.51281
Epoch 3/100


...

213/213 [==============================] - ETA: 7s - loss: 0.3924 - acc: 0.882 - ETA: 6s - loss: 0.3923 - acc: 0.882 - ETA: 5s - loss: 0.3936 - acc: 0.881 - ETA: 3s - loss: 0.3927 - acc: 0.881 - ETA: 2s - loss: 0.3922 - acc: 0.881 - ETA: 1s - loss: 0.3914 - acc: 0.882 - 301s 1s/step - loss: 0.3913 - acc: 0.8823 - val_loss: 0.4746 - val_acc: 0.8568

Epoch 00027: val_loss did not improve from 0.46828
Epoch 28/100


206/213 [============================>.] - ETA: 2:34 - loss: 0.4429 - acc: 0.937 - ETA: 2:33 - loss: 0.4181 - acc: 0.906 - ETA: 2:34 - loss: 0.4845 - acc: 0.854 - ETA: 2:33 - loss: 0.4595 - acc: 0.859 - ETA: 2:32 - loss: 0.4621 - acc: 0.862 - ETA: 2:31 - loss: 0.4597 - acc: 0.854 - ETA: 2:31 - loss: 0.4245 - acc: 0.875 - ETA: 2:30 - loss: 0.4675 - acc: 0.851 - ETA: 2:29 - loss: 0.4581 - acc: 0.847 - ETA: 2:28 - loss: 0.4481 - acc: 0.856 - ETA: 2:28 - loss: 0.4453 - acc: 0.852 - ETA: 2:27 - loss: 0.4339 - acc: 0.859 - ETA: 2:26 - loss: 0.4435 - acc: 0.855 - ETA: 2:25 - loss: 0.4418 - acc: 0.861 - ETA: 2:25 - loss: 0.4609 - acc: 0.845 - ETA: 2:24 - loss: 0.4465 - acc: 0.855 - ETA: 2:23 - loss: 0.4397 - acc: 0.856 - ETA: 2:23 - loss: 0.4298 - acc: 0.864 - ETA: 2:23 - loss: 0.4303 - acc: 0.865 - ETA: 2:33 - loss: 0.4177 - acc: 0.871 - ETA: 2:37 - loss: 0.4165 - acc: 0.875 - ETA: 2:41 - loss: 0.4042 - acc: 0.877 - ETA: 2:45 - loss: 0.3997 - acc: 0.883 - ETA: 2:49 - loss: 0.3947 - acc: 0.888

213/213 [==============================] - ETA: 7s - loss: 0.4101 - acc: 0.875 - ETA: 6s - loss: 0.4096 - acc: 0.875 - ETA: 5s - loss: 0.4085 - acc: 0.876 - ETA: 3s - loss: 0.4103 - acc: 0.876 - ETA: 2s - loss: 0.4097 - acc: 0.876 - ETA: 1s - loss: 0.4092 - acc: 0.876 - 305s 1s/step - loss: 0.4084 - acc: 0.8768 - val_loss: 0.4721 - val_acc: 0.8542

Epoch 00028: val_loss did not improve from 0.46828
Epoch 29/100


206/213 [============================>.] - ETA: 2:37 - loss: 0.5016 - acc: 0.875 - ETA: 2:36 - loss: 0.6170 - acc: 0.843 - ETA: 2:34 - loss: 0.5263 - acc: 0.833 - ETA: 2:33 - loss: 0.4880 - acc: 0.843 - ETA: 2:32 - loss: 0.5250 - acc: 0.850 - ETA: 2:32 - loss: 0.5183 - acc: 0.864 - ETA: 2:31 - loss: 0.4986 - acc: 0.857 - ETA: 2:30 - loss: 0.4917 - acc: 0.851 - ETA: 2:29 - loss: 0.4739 - acc: 0.854 - ETA: 2:29 - loss: 0.4600 - acc: 0.862 - ETA: 2:28 - loss: 0.4414 - acc: 0.869 - ETA: 2:27 - loss: 0.4223 - acc: 0.875 - ETA: 2:26 - loss: 0.4226 - acc: 0.875 - ETA: 2:26 - loss: 0.4127 - acc: 0.875 - ETA: 2:25 - loss: 0.4048 - acc: 0.879 - ETA: 2:24 - loss: 0.4069 - acc: 0.882 - ETA: 2:24 - loss: 0.4058 - acc: 0.882 - ETA: 2:23 - loss: 0.4170 - acc: 0.878 - ETA: 2:22 - loss: 0.4144 - acc: 0.878 - ETA: 2:23 - loss: 0.4157 - acc: 0.878 - ETA: 2:29 - loss: 0.4112 - acc: 0.881 - ETA: 2:32 - loss: 0.4030 - acc: 0.886 - ETA: 2:36 - loss: 0.3987 - acc: 0.891 - ETA: 2:40 - loss: 0.4039 - acc: 0.882

213/213 [==============================] - ETA: 7s - loss: 0.4114 - acc: 0.876 - ETA: 6s - loss: 0.4121 - acc: 0.876 - ETA: 5s - loss: 0.4116 - acc: 0.876 - ETA: 3s - loss: 0.4109 - acc: 0.876 - ETA: 2s - loss: 0.4100 - acc: 0.877 - ETA: 1s - loss: 0.4116 - acc: 0.876 - 300s 1s/step - loss: 0.4104 - acc: 0.8774 - val_loss: 0.4684 - val_acc: 0.8594

Epoch 00029: val_loss did not improve from 0.46828


In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if 'block_8_10' in layer.name:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
    



In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-5), metrics=['accuracy'])



In [ ]:
checkpointer = ModelCheckpoint('quicksign_inception_resnet++.h5', monitor='val_loss', save_best_only=True, verbose=1)
earlystopper = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
model.fit_generator(train_generator, epochs=50, validation_data=validation_generator, verbose=1,
                    callbacks=[checkpointer, earlystopper])

In [39]:
test_data_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_data_gen.flow_from_directory('data/test/',
                                                    target_size=(512, 512),
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                  shuffle=False)

Found 13153 images belonging to 1 classes.


In [41]:
preds = model.predict_generator(test_generator, verbose=1)

206/206 [==============================] - ETA: 13:5 - ETA: 12:1 - ETA: 11:4 - ETA: 11:2 - ETA: 11:1 - ETA: 11:0 - ETA: 10:5 - ETA: 10:5 - ETA: 10:4 - ETA: 10:4 - ETA: 10:3 - ETA: 10:3 - ETA: 10:2 - ETA: 10:2 - ETA: 10:1 - ETA: 10:1 - ETA: 10:1 - ETA: 10:0 - ETA: 10:0 - ETA: 10:0 - ETA: 9:5 - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6

In [42]:
preds.shape

(13153, 8)

In [43]:
import numpy as np
im_names = np.array(os.listdir('data/test'+'\\test_stg1'))

In [44]:
im_names = ['test_stg2/'+name if 'image' in name else name for name in im_names ]

In [45]:
import pandas as pd

In [46]:
df_names = pd.DataFrame({'image': im_names})

In [47]:
df_preds = pd.DataFrame(data=preds, columns=['ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT'])

In [48]:
df_submission = pd.concat([df_names, df_preds], axis=1)

In [49]:
df_submission.to_csv('submission_512_2.csv', index=False)

In [50]:
df_submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_00001.jpg,0.398164,0.047600,0.000087,0.001112,0.350479,0.012536,0.021999,0.168023
1,test_stg2/image_00002.jpg,0.488148,0.002543,0.003179,0.000232,0.450862,0.009787,0.018434,0.026816
2,test_stg2/image_00003.jpg,0.805502,0.001534,0.000842,0.000070,0.176604,0.011650,0.000154,0.003644
3,test_stg2/image_00004.jpg,0.329028,0.053449,0.121039,0.047917,0.050144,0.063259,0.051354,0.283810
4,test_stg2/image_00005.jpg,0.972795,0.004857,0.000636,0.000028,0.001979,0.004787,0.005128,0.009789


In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(conv_base).create(prog='dot', format='svg'))